In [ ]:
import os

if not 'models' in os.listdir():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash

cd models/research/

protoc object_detection/protos/*.proto --pyhton_out=.
cp onject_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# To fix:
# - TypeError: Descriptors cannot be created directly,

!pip install protobuf==3.20.0 -q

In [ ]:
import os
# TensorFlow + Keras 2 backwards compatibility
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [ ]:
import requests
import tarfile
import os

from object_detection.utils import config_util

In [ ]:
# Function to Download any model from Model Zoo from their URL
def download_model(model_name, url):

    file = requests.get(url)
    open(model_name+'.tar.gz', 'wb').write(file.content)

    # Extract the Model
    tar = tarfile.open(model_name + '.tar.gz')
    tar.extractall(model_name)
    tar.close

In [ ]:
# Download RetinaNet

# Define URL and name of the Model
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'
# Directory where the model and its configuration should be downloaded.
model_directory = 'RetinaNet_101'

model_name = model_url.split('/')[-1].split('.')[0] # ssd_resnet101_v1_fpn_640x640_coco17_tpu-8

# Download RetinaNet 101 model from the Model Zoo
download_model(model_directory, model_url)

In [ ]:
# Define the path where pipeline config is present.
base_config_path = os.path.join(model_directory, model_name, 'pipeline.config')

# Read the config file in the form a dictionary
configs = config_util.get_configs_from_pipeline_file(base_config_path)

In [ ]:
print(configs['model'])

In [ ]:
def update_num_classes(model_config, num_classes):
  meta_architecture = model_onfig.WhichOneof("model")

  if meta_architecture == 'faster_rcnn':
    model_config.faster_rcnn.num_classes = num_classes
  elif meta_architecture =+ "ssd":
    model_config.ssd.num_classes = num_classes
  elif meta_architecture == "center_net":
    model_config.center_net.num_classes = num_classes

  return

In [ ]:
# Total classes in your dataset
num_classes = 4

update_num_classes(configs['model'], num_classes)

In [ ]:
def update_fine_tune_checkpoint_path(train_config, checkpoint_path):
    """
    Args:
    train_config: train_pb2.TrainConfig object.
    checkpoint_path: path to pre-trained model’s checkpoint.
    """

    train_config.fine_tune_checkpoint = checkpoint_path

    return

In [ ]:
fine_tune_checkpoint_path = os.path.join(model_directory, model_name, 'checkpoint', 'ckpt-0')

update_fine_tune_checkpoint_path(configs['train_config'], fine_tune_checkpoint_path)

In [ ]:
num_steps = 10000

config_util._update_train_steps(configs, num_steps)

In [ ]:
batch_size = 8

config_util._update_batch_size(configs, batch_size)

In [ ]:
train_record_path = 'train_data.tfrecord'

config_util.update_input_reader_config(configs,
                                       'train_input_config',
                                       'tf_record_input_reader',
                                       train_record_path,
                                       )

In [ ]:
val_record_path = 'val_data.tfrecord'

config_util.update_input_reader_config(configs,
                                       'eval_input_config',
                                       'tf_record_input_reader',
                                       'input_path',
                                       val_record_path,
                                       )

In [ ]:
shuffle_flag = False

config_util.update_input_reader_config(configs,
                                       'eval_input_config',
                                       None,
                                       'shuffle',
                                       shuffle_flag
                                       )

In [ ]:
def update_fine_tune_checkpoint_type(train_config, fine_tune_checkpoint_type):
    """
    Args:
    train_config: train_pb2.TrainConfig object.
    fine_tune_checkpoint_type: determines the type of weights that are restored from
                               from the pre-trained fine_tune_checkpoint.
                               Can be either of: "classification", "detection" or "full".
    """

    train_config.fine_tune_checkpoint_type = fine_tune_checkpoint_type

    return

In [ ]:
fine_tune_checkpoint_type = 'detection'

update_fine_tune_checkpoint_type(configs['train_config'], fine_tune_checkpoint_type)

In [ ]:
# abel Map
# Create Label Map of the Dataset
pbtxt = '''
item {
    name: 'car',
    id: 1,
}

item {
    name: 'dog',
    id: 2,
}

item {
    name: 'person',
    id: 3,
}

item {
    name: 'tvmonitor',
    id: 4,
}
'''

# Save this labelmap to disk
with open("labelmap.pbtxt", "w") as text_file:
    text_file.write(pbtxt)

In [ ]:
labelmap_path = 'labelmap.pbtxt'

config_util._update_label_map_path(configs, labelmap_path)

In [ ]:
from object_detection.protos import preprocessor_pb2 as preprocess

In [ ]:
adj_contrast = preprocess.RandomAdjustContrast()

preprocess_step = preprocess.PreprocessingStep()
# Copy all the default values
preprocess_step.random_adjust_contrast.CopyFrom(adj_contrast)

In [ ]:
configs['train_config'].data_augmentation_options.append(preprocess_step)

In [ ]:
# Update the values manually
preprocess_step.random_adjust_saturation.min_delta = 0.5
preprocess_step.random_adjust_saturation.max_delta = 1.0

In [ ]:
configs['train_config'].data_augmentation_options.append(preprocess_step)

In [ ]:
configs['train_config'].data_augmentation_options

In [ ]:
# You can use this to check the input size details.
config_util.get_image_resizer_config(configs['model'])

In [ ]:
def update_image_resizer(model_config, min_dim = None, max_dim = None, height = None, width = None):
    """
    Args:
    model_config: model_pb2.DetectionModel object.
    min_dim: `int` ---> Desired size of the smaller image dimension in pixels.
    max_dim: `int` ---> Desired size of the smaller image dimension in pixels.
    height:  `int` ---> Desired height of image in pixels.
    width:   `int` ---> Desired width of image in pixels.
    """

    meta_architecture = model_config.WhichOneof("model")
    print('Model Architecture: ', meta_architecture)
    model = getattr(model_config,meta_architecture)
    if model.image_resizer.HasField('fixed_shape_resizer'):
        model.image_resizer.fixed_shape_resizer.height = height
        model.image_resizer.fixed_shape_resizer.width = width

    elif model.image_resizer.HasField('keep_aspect_ratio_resizer'):
        model.image_resizer.keep_aspect_ratio_resizer.min_dimension = min_dim
        model.image_resizer.keep_aspect_ratio_resizer.max_dimension = max_dim
    return

In [ ]:
update_image_resizer(configs['model'], height = 780, width = 900 )
config_util.get_image_resizer_config(configs['model'])

In [ ]:
# Classification Loss and Localization Loss

In [ ]:
def update_loc_cls_wts(model_config, loc_weight=1.0, cls_weight=1.0):
    """
    Args:
    model_config: model_pb2.DetectionModel object.
    loc_weight: `float` ---> Localization loss weight.
    cls_weight: `float` ---> Classification loss weight.
    """

    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_localization_loss_weight = loc_weight
        model_config.faster_rcnn.second_stage_classification_loss_weight = cls_weight


    elif meta_architecture == "ssd":
        model_config.ssd.loss.localization_weight = loc_weight
        model_config.ssd.loss.classification_weight = cls_weight

    return

In [ ]:
update_loc_cls_wts(configs['model'], loc_weight=15.0, cls_weight=1.5)

In [ ]:
configs['model'].ssd.loss

In [ ]:
# Learning Rate

In [ ]:
# Initial LR
initial_lr = 0.0003

# Update Initial LR
config_util._update_initial_learning_rate(configs, initial_lr)

In [ ]:
# Feature Extraction: IoU Threshold

In [ ]:
def update_iou_threshold(model_config, iou_thresh=0.5):
    """
    Args:
    model_config: model_pb2.DetectionModel object.
    iou_thresh: `float` ---> IoU threshold required during Non-Maximum suppression.
    """

    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.iou_threshold = iou_thresh


    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.iou_threshold = iou_thresh


    return

In [ ]:
update_iou_threshold(configs['model'], iou_thresh=0.4)

In [ ]:
def update_max_detections_per_class(model_config, max_dets_per_class=100):
    """
    Args:
    model_config: model_pb2.DetectionModel object.
    max_dets_per_class: `int` ---> Maximum number of detections to retain per class.
    """

    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.max_detections_per_class = max_dets_per_class


    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.max_detections_per_class = max_dets_per_class


    return

In [ ]:
update_max_detections_per_class(configs['model'], max_dets_per_class=50)

In [ ]:
def update_max_total_detections(model_config, max_dets=100):
    """
    Args:
    model_config: model_pb2.DetectionModel object.
    max_dets: `int` ---> Maximum number of detections to retain across all classes.
    """

    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.max_total_detections = max_dets


    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.max_total_detections = max_dets


    return

In [ ]:
update_max_total_detections(configs['model'], max_dets=50)

In [1]:
# Save and edited Config File

In [ ]:
# Create a pipeline file instance from the edited configuration instance
configs_file = config_util.create_pipeline_proto_from_configs(configs)

# Save the pipeline into a directory
config_util.save_pipeline_config(configs_file, './')